## Content

1. [Methods](#Methods)
2. [Results](#Results)

    2.1 [Black-Scholes](#Black-Scholes)

    2.2 [Classical Monte Carlo](#Classical-Monte-Carlo)
    
    2.3 [Quantum-Classical Monte Carlo](#Quantum-Classical-Monte-Carlo)

3. [European put option](#European-put-option)


### Methods

In [469]:
import numpy as np
from numpy import percentile
import datetime as dt
import scipy.stats as st
import pandas as pd
from qiskit.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit_finance.applications.estimation import EuropeanCallPricing
from qiskit_algorithms import MaximumLikelihoodAmplitudeEstimation 

#### Black-Scholes formula

In [470]:
def black_scholes_european_option(initial_price, dividends_rate, risk_free_rate, volatility, initial_time, strike, expiry_date, type):
    """
    Black-Scholes formula for calculating a European call or put option premium, where the underlying 
    follows a geometric Brownian motion (GBM) with constant parameters.

    Parameters
    ----------
    initial_price: float
    dividends_rate: float
    risk_free_rate: float
    volatility: float
    strike: float
    initial_time: float
    expiry_date: float
    type: int
        1 for a call option and -1 for a put option

    Returns
    -------
    option_premium: float
    """

    drift = risk_free_rate - dividends_rate
    d1 = (np.log(initial_price / strike) + (drift + 0.5 * volatility**2) * (expiry_date - initial_time)) / (volatility * np.sqrt(expiry_date - initial_time))
    d2 = d1 - volatility * np.sqrt(expiry_date - initial_time)

    phi1 = st.norm.cdf(type * d1)
    phi2 = st.norm.cdf(type * d2)

    option_premium = type * initial_price * np.exp(-dividends_rate * (expiry_date - initial_time)) * phi1  - type * np.exp(-risk_free_rate * (expiry_date - initial_time)) * strike * phi2  
    
    return option_premium


#### Classical Monte Carlo simulation

In [471]:
def classical_monte_carlo_gmb(nb_simulations, initial_price, drift, volatility, initial_time, expiry_date, seed):
    """
    Classical Monte Carlo simulation of a geometric Brownian motion (GBM) with constant parameters.

    Parameters
    ----------
    nb_simulations: int
    initial_price: float
    drift: float
    volatility: float
    initial_time: float
    expiry_date: float
    seed: int


    Returns
    -------
    results: list
    """
    
    results = []
    np.random.seed(seed)
    for i in range(0, nb_simulations):  
        
        increment = 1
        steps = round((expiry_date-initial_time)/increment)
        random_variables = np.random.normal(0, 1, steps)
        time_discretization = np.linspace(0, (expiry_date-initial_time), steps)
        brownian_motions = np.cumsum(random_variables) * np.sqrt(increment)
        geometric_brownian_motions = (drift - 0.5 * volatility ** 2) * time_discretization + volatility * brownian_motions
        simulated_prices = initial_price * np.exp(geometric_brownian_motions)        
        results.append(simulated_prices[-1])

    return results


def discounted_european_payoff(simulation, risk_free_rate, strike, initial_time, expiry_date, type):
    """
    European option premium calculation from classical Monte Carlo simulated prices.

    Parameters
    ----------
    simulation: list
    risk_free_rate: float
    strike: float
    initial_time: float
    expiry_date: float
    
    Returns
    -------
    option_premium_approximation: float
    """
    
    payoff = lambda s: max(s - strike, 0) if type == 1 else min(strike - s, 0) 
    simulated_payoff = [payoff(s) for s in simulation]

    option_premium_approximation = np.exp(-risk_free_rate * (expiry_date - initial_time)) * np.average(simulated_payoff)

    return option_premium_approximation
    

#### $(\Delta, n, \alpha)$ discretization method 

In [472]:
def distribution(risk_free_rate, volatility, initial_price, expiry_date, initial_time, dividend):
    """
    Log-normal distribution parameters.

    Parameters
    ----------
    risk_free_rate: float
    volatility: float
    initial_time: float
    expiry_date: float
    initial_price: float
    dividend: float

    Returns
    -------
    log_normal_stats: list
        [mu, sigma, mean, stddev]
    """

    drift = risk_free_rate - dividend
    mu = (drift - 0.5 * volatility**2) * (expiry_date - initial_time) + np.log(initial_price)
    sigma = volatility * np.sqrt(expiry_date - initial_time)
    mean = np.exp(mu + 0.5 * sigma**2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
    stddev = np.sqrt(variance)

    log_normal_stats = [mu, sigma, mean, stddev]

    return log_normal_stats


def informed_discretization(parameters, delta, nb_qubits, alpha, gamma=0.0, heuristic=False):
    """
    Log-normal discretization via the (Delta, n, alpha) method or heuristic truncation.

    Parameters
    ----------
    parameters: list
    delta: float
    nb_qubits: int
    alpha: int
        Bigger or equal to 1
    gamma: float
    heuristic: bool
        Optional, default False

    Returns
    -------
    discrete_distribution: list
        [std_scaling, x, y]
    """

    mu = parameters[0]
    sigma = parameters[1]
    mean = parameters[2]
    stddev = parameters[3]
            
    if heuristic:
        std_scaling = gamma
       
    elif not heuristic:              
        # Scaling factor method
        std_scaling = (delta * (2**nb_qubits - 1) - mean) / stddev

        if std_scaling < mean / stddev:
            print(f"Invalid scaling factor value = {std_scaling}") 
            return 0

    low = np.maximum(0, mean - std_scaling * stddev)
    high = mean + std_scaling * stddev
        
    uncertainty_model = LogNormalDistribution(nb_qubits, mu=mu, sigma=sigma**2, bounds=(low, high))
    x = uncertainty_model.values
    y = uncertainty_model.probabilities

    if sum(y) < 1-10**(-alpha):
        print(f"Discrete density = {sum(y)} < {1-10**(-alpha)}")
        return 0

    discrete_distribution = [std_scaling, x, y]

    return discrete_distribution

#### FVA and AVA calculation

In [473]:
def valuation_adjustments(data, nb_options, precision, percent, eta):
    """
    Fair and additional valuation adjustment calculation.

    Parameters
    ----------
    data: pd.DataFrame
    nb_options: int
    precision: int
    percent: int
        range [0, 100] 
    eta: float
        range ]0,1[

    Returns
    -------
    va_results: pd.DataFrame
    """

    va_results = pd.DataFrame()
    for i in range(len(data)):
        np = round(data[i].loc[data[i]['Dividend'] == 0.0, data[i].columns[1]].values[-1], precision)
        fv = round(data[i].loc[data[i]['Dividend'] == 0.02, data[i].columns[1]].values[-1], precision)
        fva = fv - np
        sample = data[i].loc[data[i]['Dividend'] > 0.02, data[i].columns[1]]
        pv =  round(percentile(sample, percent), precision) 
        ava = fv - pv
   
        va_results.loc[i, 'Method'] = data[i].columns[1]
        va_results.loc[i, 'FVA (EUR)'] = nb_options * fva
        va_results.loc[i, 'PV (EUR)'] = pv
        va_results.loc[i, 'AVA (EUR)'] = (1 - eta) * nb_options * ava

    return va_results        

### Results

#### Inputs

In [491]:
# European call option
type = 1
initial_price = 60
risk_free_rate = 0.05
volatility = 0.2
strike = 65
initial_time = 0
expiry_date = 5

# Portfolio size
nb_options = 500

# Classical MC simulations
M = 10**5

# Discretization-informed
delta = 9
num_qubits = 6
alpha = 15

# MLQAE algorithm
eval = 5
shots = int(np.ceil(M / (sum([(2*2**m + 1) for m in range(0,eval)]) + 1)))

# Valuation adjustments
dividend = [0.0, 0.02, 0.0275, 0.035, 0.0425, 0.05]
dec_prec = 2
percent = 10
eta = 0.4

#### Black-Scholes

In [492]:
df_results = pd.DataFrame()
for d in range(0, len(dividend)):
    
    exact_option_premium = black_scholes_european_option(
                                                        initial_price,
                                                        dividend[d], 
                                                        risk_free_rate,
                                                        volatility,
                                                        initial_time,
                                                        strike, 
                                                        expiry_date, 
                                                        type)

    df_results.loc[d, 'Dividend'] = dividend[d]
    df_results.loc[d, 'BS'] = exact_option_premium

df_results

,Dividend,BS
0,0.0000,15.160246
1,0.0200,11.226911
2,0.0275,9.963217
3,0.0350,8.807617
4,0.0425,7.755144
5,0.0500,6.800612


In [493]:
valuation_adjustments([df_results], nb_options, dec_prec, percent, eta)

,Method,FVA (EUR),PV (EUR),AVA (EUR)
0,BS,-1965.0,7.09,1242.0


#### Classical Monte Carlo

In [477]:
df_mc_results = pd.DataFrame()
for nb_sim in [M]:
    for d in range(0, len(dividend)):
        
        mu = risk_free_rate - dividend[d]
        begin = dt.datetime.now()
        simulations_results = classical_monte_carlo_gmb(
                                                        nb_sim,
                                                        initial_price,
                                                        mu,
                                                        volatility,
                                                        initial_time, 
                                                        expiry_date,
                                                        seed=int(1000*d + 1))
        approx_option_premium = discounted_european_payoff(
                                                        simulations_results,
                                                        risk_free_rate, strike, 
                                                        initial_time, 
                                                        expiry_date, 
                                                        type)
        end = dt.datetime.now()
        
        df_mc_results.loc[d, 'Dividend'] = dividend[d]
        df_mc_results.loc[d, f'MC {nb_sim}'] = approx_option_premium
        df_mc_results.loc[d, f'MC {nb_sim} acc.'] = np.format_float_scientific(round(approx_option_premium - df_results.loc[d, 'BS'], 5))

df_mc_results

,Dividend,MC 100000,MC 100000 acc.
0,0.0000,15.231345,7.11e-02
1,0.0200,11.274427,4.752e-02
2,0.0275,9.952912,-1.03e-02
3,0.0350,8.736246,-7.137e-02
4,0.0425,7.658453,-9.669e-02
5,0.0500,6.819233,1.862e-02


In [478]:
valuation_adjustments([df_mc_results], nb_options, dec_prec, percent, eta)

,Method,FVA (EUR),PV (EUR),AVA (EUR)
0,MC 100000,-1980.0,7.07,1260.0


#### Quantum-Classical Monte Carlo

#### $\gamma=4$

In [494]:
df_discrete_results_trunc = pd.DataFrame()
for d in range(len(dividend)):  
    gamma = 4
    param = distribution(risk_free_rate, volatility, initial_price, expiry_date, initial_time, dividend[d])
    disc_dist = informed_discretization(param, delta, num_qubits, alpha, gamma, heuristic=True)
    x = disc_dist[1]
    y = disc_dist[2]
    payoff = [max(type * i - type * strike, 0) * j for i,j in zip(x,y)]
    premium_approximation = np.exp(-risk_free_rate * (expiry_date - initial_time)) * sum(payoff)

    df_discrete_results_trunc.loc[d, 'Dividend'] =  dividend[d] 
    df_discrete_results_trunc.loc[d, 'Estimation'] =  premium_approximation  
    df_discrete_results_trunc.loc[d, 'Accuracy'] = premium_approximation - df_results.loc[d, 'BS']
    df_discrete_results_trunc.loc[d, 'Gamma'] =  gamma
    

df_discrete_results_trunc['Accuracy'] = df_discrete_results_trunc['Accuracy'].apply(lambda x: np.format_float_scientific(round(x, 5)))
df_discrete_results_trunc

,Dividend,Estimation,Accuracy,Gamma
0,0.0000,14.541366,-6.1888e-01,4.0
1,0.0200,10.676527,-5.5038e-01,4.0
2,0.0275,9.432440,-5.3078e-01,4.0
3,0.0350,8.296683,-5.1093e-01,4.0
4,0.0425,7.273803,-4.8134e-01,4.0
5,0.0500,6.344800,-4.5581e-01,4.0


#### $(\Delta=9, n=6, \alpha=15)$ discretization method 

In [495]:
df_discrete_results = pd.DataFrame()
for d in range(len(dividend)):

    param = distribution(risk_free_rate, volatility, initial_price, expiry_date, initial_time, round(dividend[d],4))
    disc_dist = informed_discretization(param, delta, num_qubits, alpha, gamma=0.0, heuristic=False)
    gamma = disc_dist[0]
    x = disc_dist[1]
    y = disc_dist[2]
    payoff = [max(type * i - type * strike, 0) * j for i,j in zip(x,y)]
    premium_approximation = np.exp(-risk_free_rate * (expiry_date - initial_time)) * sum(payoff)

    df_discrete_results.loc[d, 'Dividend'] =  dividend[d] 
    df_discrete_results.loc[d, 'Discrete'] =  premium_approximation  
    df_discrete_results.loc[d, 'Accuracy'] = premium_approximation - df_results.loc[d, 'BS']
    df_discrete_results.loc[d, 'Gamma'] =  gamma
    

df_discrete_results['Accuracy'] = df_discrete_results['Accuracy'].apply(lambda x: np.format_float_scientific(round(x, 5)))
df_discrete_results

,Dividend,Discrete,Accuracy,Gamma
0,0.0000,15.160992,7.5e-04,13.515835
1,0.0200,11.226994,8.e-05,15.160822
2,0.0275,9.962969,-2.5e-04,15.821357
3,0.0350,8.807026,-5.9e-04,16.507132
4,0.0425,7.754217,-9.3e-04,17.219112
5,0.0500,6.799369,-1.24e-03,17.958299


In [481]:
valuation_adjustments([df_discrete_results], nb_options, dec_prec, percent, eta)

,Method,FVA (EUR),PV (EUR),AVA (EUR)
0,Discrete,-1965.0,7.09,1242.0


#### Maximum Likelihood Quantum Amplitude Estimation

In [482]:
df_mlqae_results = pd.DataFrame()
for d in range(len(dividend)):

    param = distribution(risk_free_rate, volatility, initial_price, expiry_date, initial_time, dividend[d])
    mu = param[0]
    sigma = param[1]
    mean = param[2]
    stddev = param[3]
    disc_dist = informed_discretization(param, delta, num_qubits, alpha, gamma=0.0, heuristic=False)
    
    gamma = df_discrete_results.loc[d,'Gamma']
    x = disc_dist[1]
    y = disc_dist[2]

    low = np.maximum(0, mean - gamma * stddev)
    high = mean + gamma * stddev

    uncertainty_model = LogNormalDistribution(num_qubits, mu=mu, sigma=sigma**2, bounds=(low, high))
    
    set_seed = dividend[d]
    mlqae = MaximumLikelihoodAmplitudeEstimation(
                                                evaluation_schedule=eval,
                                                sampler=Sampler(options={"shots": shots, "seed": int(1000*set_seed + 1)}))
    
    c_scaling = [0.078, 0.061, 0.089, 0.073, 0.074, 0.069]        
    european_call_pricing = EuropeanCallPricing(
                                                num_state_qubits=num_qubits,
                                                strike_price=strike,
                                                rescaling_factor=c_scaling[d],
                                                bounds=(low, high),
                                                uncertainty_model=uncertainty_model)
        
    problem = european_call_pricing.to_estimation_problem()
    mlqae_result = mlqae.estimate(problem)        
    discount_factor = np.exp(-risk_free_rate * (expiry_date - initial_time))
    mlqae_premium = discount_factor * european_call_pricing.interpret(mlqae_result)

    df_mlqae_results.loc[d, 'Dividend'] =  dividend[d] 
    df_mlqae_results.loc[d, 'MLQAE'] =  mlqae_premium  
    df_mlqae_results.loc[d, 'Accuracy'] = mlqae_premium - df_discrete_results.loc[d, 'Discrete']
    df_mlqae_results.loc[d, 'c'] =  c_scaling[d]

df_mlqae_results['Accuracy'] = df_mlqae_results['Accuracy'].apply(lambda x: np.format_float_scientific(round(x, 5)))

df_mlqae_results

,Dividend,MLQAE,Accuracy,c
0,0.0000,15.202171,4.118e-02,0.078
1,0.0200,11.242246,1.525e-02,0.061
2,0.0275,9.916518,-4.645e-02,0.089
3,0.0350,8.829691,2.266e-02,0.073
4,0.0425,7.800208,4.599e-02,0.074
5,0.0500,6.110878,-6.8849e-01,0.069


In [483]:
valuation_adjustments([df_mlqae_results], nb_options, dec_prec, percent, eta)

,Method,FVA (EUR),PV (EUR),AVA (EUR)
0,MLQAE,-1980.0,6.62,1386.0


In [484]:
data = [df_results, df_discrete_results, df_mc_results, df_mlqae_results]
results = valuation_adjustments(data, nb_options, dec_prec, percent, eta)
results

,Method,FVA (EUR),PV (EUR),AVA (EUR)
0,BS,-1965.0,7.09,1242.0
1,Discrete,-1965.0,7.09,1242.0
2,MC 100000,-1980.0,7.07,1260.0
3,MLQAE,-1980.0,6.62,1386.0


### European put option

#### Inputs

In [499]:
# European put option
type = -1
initial_price = 90
risk_free_rate = 0.02
dividend = 0.0
volatility = [0.1, 0.5, 0.7]
strike = 75
initial_time = 0
expiry_date = 10

# Discretization-informed
delta = 4
num_qubits = 6
alpha = 15

#### Black-Scholes

In [500]:
df_put_results = pd.DataFrame()
for v in range(0, len(volatility)):
    
    exact_option_premium = black_scholes_european_option(
                                                        initial_price,
                                                        dividend, 
                                                        risk_free_rate,
                                                        volatility[v],
                                                        initial_time,
                                                        strike, 
                                                        expiry_date, 
                                                        type)

    df_put_results.loc[v, 'Vol'] = volatility[v]
    df_put_results.loc[v, 'BS'] = exact_option_premium

df_put_results

,Vol,BS
0,0.1,1.283817
1,0.5,29.914820
2,0.7,41.618683


#### $\gamma=3$

In [501]:
gamma = 3

df_put_discrete_results_trunc = pd.DataFrame()
for d in range(len(volatility)):  
    param = distribution(risk_free_rate, volatility[d], initial_price, expiry_date, initial_time, dividend)
    disc_dist = informed_discretization(param, delta, num_qubits, alpha, gamma, heuristic=True)
    x = disc_dist[1]
    y = disc_dist[2]
    payoff = [max(type * i - type * strike, 0) * j for i,j in zip(x,y)]
    premium_approximation = np.exp(-risk_free_rate * (expiry_date - initial_time)) * sum(payoff)

    df_put_discrete_results_trunc.loc[d, 'Vol'] =  volatility[d] 
    df_put_discrete_results_trunc.loc[d, 'Estimation'] =  premium_approximation  
    df_put_discrete_results_trunc.loc[d, 'Accuracy'] = premium_approximation - df_put_results.loc[d, 'BS']
    df_put_discrete_results_trunc.loc[d, 'Gamma'] =  gamma
    

df_put_discrete_results_trunc['Accuracy'] = df_put_discrete_results_trunc['Accuracy'].apply(lambda x: np.format_float_scientific(round(x, 5)))
df_put_discrete_results_trunc

,Vol,Estimation,Accuracy,Gamma
0,0.1,1.296792,1.298e-02,3.0
1,0.5,21.154271,-8.76055e+00,3.0
2,0.7,4.872436,-3.674625e+01,3.0


#### $(\Delta=4, n=6, \alpha=15)$ discretization method 

In [503]:
df_put_discrete_results = pd.DataFrame()
for d in range(len(volatility)):

    param = distribution(risk_free_rate, volatility[d], initial_price, expiry_date, initial_time, dividend)
    disc_dist = informed_discretization(param, 4, num_qubits, alpha, gamma=0.0, heuristic=False)
    gamma = disc_dist[0]
    x = disc_dist[1]
    y = disc_dist[2]
    payoff = [max(type * i - type * strike, 0) * j for i,j in zip(x,y)]
    premium_approximation = np.exp(-risk_free_rate * (expiry_date - initial_time)) * sum(payoff)

    df_put_discrete_results.loc[d, 'Vol'] =  volatility[d] 
    df_put_discrete_results.loc[d, 'Discrete'] =  premium_approximation  
    df_put_discrete_results.loc[d, 'Accuracy'] = premium_approximation - df_put_results.loc[d, 'BS']
    df_put_discrete_results.loc[d, 'Gamma'] =  gamma
    

df_put_discrete_results['Accuracy'] = df_put_discrete_results['Accuracy'].apply(lambda x: np.format_float_scientific(round(x, 5)))
df_put_discrete_results

,Vol,Discrete,Accuracy,Gamma
0,0.1,1.288114,4.3e-03,3.985333
1,0.5,31.697102,1.78228e+00,0.386494
2,0.7,38.648930,-2.96975e+00,0.111947
